In [1]:
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline
import ROOT
%jsroot on

Welcome to JupyROOT 6.22/06


In [2]:
def new_rechits_all(df_):
    
    rechit_x = []
    rechit_y = []
    rechit_z = []
    rechit_ene = []
    
    
    rec_lay = df_.comb_rechit_z_trimAhcal
    
    rec_x = df_.comb_rechit_x_trimAhcal[rec_lay<55.0]
    rec_y = df_.comb_rechit_y_trimAhcal[rec_lay<55.0]
    rec_z = df_.comb_rechit_z_trimAhcal[rec_lay<55.0]
    rec_ene = df_.rechitEn_trimAhcal[rec_lay<55.0]
        
    new_cell_idxs = []
    new_rechit_x = []
    new_rechit_y = []
    new_rechit_z = []
    new_rechit_ene = []

    for rec_i in range(rec_x.size):


            cell_idx = np.array([rec_x[rec_i], rec_y[rec_i]]).tolist()


            if cell_idx not in new_cell_idxs:

                new_cell_idxs.append(cell_idx)

                new_rechit_x.append(rec_x[rec_i])
                new_rechit_y.append(rec_y[rec_i])
                new_rechit_z.append(53.9035)
                #new_rechit_z.append(13.8774)
                new_rechit_ene.append(rec_ene[rec_i])

            else: new_rechit_ene[new_cell_idxs.index(cell_idx)] += rec_ene[rec_i]
                
    rechit_x += new_rechit_x
    rechit_y += new_rechit_y
    rechit_z += new_rechit_z
    rechit_ene += new_rechit_ene
    
    
    
    
    
    rec_x = df_.comb_rechit_x_trimAhcal[rec_lay>55.0]
    rec_y = df_.comb_rechit_y_trimAhcal[rec_lay>55.0]
    rec_z = df_.comb_rechit_z_trimAhcal[rec_lay>55.0]
    rec_ene = df_.rechitEn_trimAhcal[rec_lay>55.0]
        
    new_rechit_x = list(rec_x)
    new_rechit_y = list(rec_y)
    new_rechit_z = list(rec_z)
    new_rechit_ene = list(rec_ene)

                
    rechit_x += new_rechit_x
    rechit_y += new_rechit_y
    rechit_z += new_rechit_z
    rechit_ene += new_rechit_ene
        
            
    
    
    return [np.array(rechit_x), np.array(rechit_y), np.array(rechit_z), np.array(rechit_ene)]

In [3]:
data = uproot.open("/eos/user/n/nkasarag/NSM_5M_TB_HGCAL_flat_pion_samples/skimmed_ntuple_sim_config22_pions_1to2M_combinedHgc_Ahc_v46.root")

lst = [ 'rechitEn_trimAhcal',
 'comb_rechit_x_trimAhcal',
 'comb_rechit_y_trimAhcal',
 'comb_rechit_z_trimAhcal',
      'trueBeamEnergy']

In [4]:
num_events = data["pion_variables_v1"].pandas.df("trueBeamEnergy", flatten=False).trueBeamEnergy.size
print(num_events)


#event_lst = [0, 100000, 200000, 300000, 400000, num_events]
event_lst = [0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, num_events]
event_lst

837795


[0, 100000, 200000, 300000, 400000, 500000, 600000, 700000, 837795]

In [5]:
for i in range(9):
    
    time1 = time.time()
    
    df = data["pion_variables_v1"].pandas.df(lst, flatten=False, entrystart=event_lst[i], entrystop=event_lst[i+1])
    df.reset_index(inplace=True)
    
    rechits = df.apply(new_rechits_all, axis=1)
    new_df = pd.DataFrame(rechits.values.tolist(), columns=["reduced_x", "reduced_y", "reduced_z", "reduced_ene"])

    
    new_df["trueBeamEnergy"] = df.trueBeamEnergy
    
    
    df_to_save = new_df
    
    
    df_to_save.to_pickle("pkl_files/1to2M/event_%sto%s_1to2M.pickle"%(str(event_lst[i]), str(event_lst[i+1])))
    
    print(i+1, "-> mins = ", (time.time()-time1)/60)


1 -> mins =  4.387083947658539
2 -> mins =  4.550169630845388
3 -> mins =  4.60330129067103
4 -> mins =  4.756057953834533
5 -> mins =  4.72608996629715
6 -> mins =  5.13712652126948
7 -> mins =  4.337847582499186
8 -> mins =  5.977793137232463


IndexError: list index out of range

In [6]:
pwd


'/eos/home-i00/n/nkasarag/SWAN_projects/HGCal_Particle_Identificatin_and_Energy_Reconstruction/Reducing_longitudinal_granularity'